# Overview

This notebook prepares landings records from the Government of Greenland, Fisheries Department, Fisheries License Control Authority, for the purposes of reproducing the results of the sea ice fishing study.

## Load Packages

In [56]:
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.covariance import EllipticEnvelope
from tqdm.notebook import tqdm
import json
from pathlib import Path

## Load and Clean Landings Records

In [57]:
landings = pd.read_csv('../data/landings_raw.csv', index_col=0, encoding = "ISO-8859-1", low_memory=False)

### Create Dictionary of Vessel Types

In [58]:
vessels = {
    'UJOLLE': None,
    'USLAEDE': None,
    'Dinghy': 'water',
    'Larger Inshore Vessel': 'water',
    'Sled': 'ice',
    'Snowmobile': 'ice',
    'ATV': None,
}
landings['vessel'] = landings.vessel_type
landings['vessel_type'] = landings.vessel.map(vessels)

### Correct Erroneous Localities Values

In [59]:
# Some localities have the wrong name
landings['sellers_locality'] = landings['sellers_locality'].replace({'Tasiusaq, Upernavik': 'Upernavik'})
landings['sellers_locality'] = landings['sellers_locality'].replace({'Kuummiit': 'Kuummiut'})
landings['sellers_locality'] = landings['sellers_locality'].replace({'Tiileqilaaq': 'Tiniteqilaaq'})

# Merge localities 
landings.loc[landings['sellers_locality'] == 'Aappilattoq Nanortalimmi', 'sellers_locality'] = 'Nanortalik'
landings.loc[landings['sellers_locality'] == 'Aappilattoq Upernavimmi', 'sellers_locality'] = 'Upernavik'
landings.loc[landings['sellers_locality'] == 'Nuussuaq, Nuuk', 'sellers_locality'] = 'Nuuk'
landings.loc[landings['sellers_locality'] == 'Nuussuaq, Upernavik', 'sellers_locality'] = 'Upernavik'
landings.loc[landings['sellers_locality'] == 'Tasiusaq Nanortalimmi', 'sellers_locality'] = 'Nanortalik'
landings.loc[landings['sellers_locality'] == 'Tasiusaq, Upernavik', 'sellers_locality'] = 'Tasiusaq'
landings.loc[landings['sellers_locality'] == 'Kuummiit', 'sellers_locality'] = 'Kuummiut'
landings.loc[landings['buyers_locality'] == 'Aappilattoq Nanortalimmi', 'buyers_locality'] = 'Nanortalik'
landings.loc[landings['buyers_locality'] == 'Aappilattoq Upernavimmi', 'buyers_locality'] = 'Upernavik'
landings.loc[landings['buyers_locality'] == 'Nuussuaq, Nuuk', 'buyers_locality'] = 'Nuuk'
landings.loc[landings['buyers_locality'] == 'Nuussuaq, Upernavik', 'buyers_locality'] = 'Upernavik'
landings.loc[landings['buyers_locality'] == 'Tasiusaq Nanortalimmi', 'buyers_locality'] = 'Nanortalik'
landings.loc[landings['buyers_locality'] == 'Tasiusaq, Upernavik', 'buyers_locality'] = 'Tasiusaq'
landings.loc[landings['buyers_locality'] == 'Kuummiit', 'buyers_locality'] = 'Kuummiut'

print(f'There are {landings.shape[0]} entries in the landings dataset.')

There are 1662470 entries in the landings dataset.


### Correct Gears

In [60]:
landings['gears'] = landings.gear_en.map({"Set gillnets": "Gill nets", "Gill nets": "Gill nets", "Longlines (not specified)": "Longlines"})

landings.gears.value_counts()


gears
Longlines    1053874
Gill nets     270563
Name: count, dtype: int64

### Drop Erroneous Records

In [61]:
landings.dropna(subset= "landing_date", inplace =True)
landings = landings.dropna(subset='num_tools')
landings = landings[landings["num_tools"] > 0]

In [62]:
landings.shape

(1630798, 25)

In [63]:
landings = landings[~((landings["gear_en"] == "Longlines") & (landings["num_tools"] < 30))]

In [64]:
landings = landings[landings["fishing_time"] > 0]

### Correct Spatial Data Values

In [65]:
# Clean up degree symbols
landings.latitude = landings.latitude.str.replace('¡', '°')
landings.longitude = landings.longitude.str.replace('¡', '°')

# Some missing data are okay, but some are an immediate disqualifier for further analysis of that row
landings.dropna(subset=['seller_id'], inplace=True)

field_codes = pd.read_csv('../data/fieldcodes.csv', index_col='fieldcode')
field_codes = field_codes.to_dict(orient='index')

def lookup_lat_lon(row):
    """ Find latitude and longitude based on field code """
    try: 
        row['latitude'] = field_codes[row['field_code']]['lat']
        row['longitude'] = field_codes[row['field_code']]['lon']
    except KeyError:
        row['latitude'] = None
        row['longitude'] = None
    return row

landings = landings.apply(lookup_lat_lon, axis='columns')

## Load Fieldcodes

In [66]:
field_code_info = pd.read_csv('../data/fieldcodes.csv', index_col='fieldcode').to_dict(orient='index')


In [67]:
landings['field_code'] = landings.field_code.str.replace('-', '')

In [68]:
def insert_zero_padding(field_code):
    try:
        return field_code[:2] + f'{int(field_code[2:]):03d}'                
    except (ValueError, TypeError):
        return field_code
    
landings['field_code'] = landings.field_code.apply(insert_zero_padding)

In [69]:
bad_field_codes = []
for field_code in landings.field_code.unique():
    if not field_code in field_code_info.keys():
        bad_field_codes.append(field_code)



In [70]:
landings = landings[~landings.field_code.isin(bad_field_codes)]

## Load Locality Information

In [71]:
locality_info = pd.read_csv('../data/localities_expanded.csv', index_col='TEKST_GL')
locality_info.dropna(subset=['KOMMUNE', 'TYPE'], inplace=True)
locality_info = locality_info.to_dict(orient='index')

def get_municipality(locality):
    try:
        return locality_info[locality]['KOMMUNE']
    except KeyError:
        return 'unknown'

def get_settlement_size(locality):
    try:
        return locality_info[locality]['TYPE']
    except KeyError:
        return 'unknown'

landings['sellers_municipality'] = landings.sellers_locality.apply(get_municipality)
landings['sellers_settlement_size'] = landings.sellers_locality.apply(get_settlement_size)

## Create Derived Variables

### Create Seasonal Year

In [72]:
landings['landing_date'] =  pd.to_datetime(landings.landing_date)
landings['seasonal_year'] = landings.landing_date.apply(lambda x: x.year - 1 if x.month < 8 else x.year)
landings = landings.query("seasonal_year > 2011 & seasonal_year < 2023")


### Calculate Catch Per Unit of Effort (CPUE)

In [73]:
def calculate_cpue(landing):
    if landing.gear_en == "Longlines":
        landing.cpue = landing.amount_in_kg / (landing.num_tools / 100) / landing.fishing_time
    elif landing.gear_en == "Gill nets":
        landing.cpue = landing.amount_in_kg / landing.num_tools / landing.fishing_time
    return landing
landings["cpue"] = None

landings = landings.apply(calculate_cpue, axis=1)
landings.shape 

(1520208, 29)

### Calculate Price Per Kilo (DKK/kg)

In [74]:
landings['ppk'] = landings.value / landings.amount_in_kg
landings.replace([np.inf, -np.inf], np.nan, inplace=True)
landings.dropna(subset=['cpue', 'ppk'], inplace=True)

### Outlier Removal

In [75]:
def is_outlier(s):
    if s.shape[0] < 3:
        return s != s
    return pd.Series(EllipticEnvelope(support_fraction=0.8).fit_predict(s.values.reshape(-1, 1)) == -1, index=s.index)

landings = landings[~landings.groupby('sellers_municipality', group_keys=False)['cpue'].apply(is_outlier)]
landings = landings[~landings.groupby('sellers_municipality', group_keys=False)['ppk'].apply(is_outlier)]
landings = landings[~landings.groupby('sellers_municipality', group_keys=False)['value'].apply(is_outlier)]

## Create Derived Dataframes

### Calculate Price Per Kilo According to Locality

In [76]:
ppk_locality = landings.groupby(by=['sellers_locality', 'seasonal_year']).ppk.mean().reset_index()

ppk_locality

ppk_locality.to_csv('../data/ppk_locality.csv')



### Ice Landings and Water Landings

In [77]:
ice_landings = landings.query('vessel_type == "ice"')
water_landings = landings.query('vessel_type == "water"')

ice_landings.to_csv('../data/ice_landings.csv')
water_landings.to_csv('../data/water_landings.csv')

### Total Daily Catch

In [78]:
total_daily_catch = ice_landings.groupby(by=['seasonal_year', 'landing_date']).amount_in_kg.sum().reset_index()

def calc_seasonal_days(row):   
    return (row['landing_date'] - dt.datetime(row['seasonal_year'], 8, 1)).days

total_daily_catch['seasonal_day'] = total_daily_catch[['landing_date', 'seasonal_year']].apply(calc_seasonal_days, axis='columns')

total_daily_catch['cumulative'] = total_daily_catch.sort_values(by='seasonal_day').groupby(by=['seasonal_year']).amount_in_kg.cumsum()

total_daily_catch.to_csv('../data/total_daily_catch.csv')

### Local Daily Catch

In [79]:
local_daily_catch = landings.query('vessel_type == "ice"').groupby(by=['seasonal_year', 'field_code', 'landing_date', 'sellers_locality']).amount_in_kg.sum().reset_index()

def calc_seasonal_days(row):   
    return (row['landing_date'] - dt.datetime(row['seasonal_year'], 8, 1)).days

local_daily_catch['seasonal_day'] = local_daily_catch[['landing_date', 'seasonal_year']].apply(calc_seasonal_days, axis='columns')

local_daily_catch['cumulative'] = local_daily_catch.sort_values(by='seasonal_day').groupby(by=['seasonal_year', 'field_code', 'sellers_locality']).amount_in_kg.cumsum()


local_daily_catch.to_csv('../data/local_daily_catch.csv')

### Local First Catch

In [80]:
local_first_catch = local_daily_catch.groupby(by=['seasonal_year', 'field_code', 'sellers_locality']).seasonal_day.min().reset_index()\
    .rename(columns={'seasonal_day': 'first_catch_day', 'seasonal_year': 'season'})

local_first_catch.to_csv('../data/local_first_catch.csv')

### Local Last Catch

In [81]:
local_last_catch = local_daily_catch.groupby(by=['seasonal_year', 'field_code', 'sellers_locality']).seasonal_day.max().reset_index()\
    .rename(columns={'seasonal_day': 'last_catch_day', 'seasonal_year': 'season'})
local_last_catch.to_csv('../data/local_last_catch.csv')

local_last_catch_dates

,season,field_code,sellers_locality,last_catch_day
0,2012,LN028,Qeqertaq,218
1,2012,LP028,Qeqertaq,230
2,2012,LP028,Saqqaq,222
3,2012,LX025,Uummannaq,252
4,2012,LX027,Niaqornat,198
...,...,...,...,...
391,2021,MX009,Innaarsuit,234
392,2021,MZ008,Innaarsuit,232
393,2021,NA008,Upernavik,264
394,2021,NB008,Upernavik,246


## Season Length (First/Last Catch Criteria)

In [82]:
total_first_catch = total_daily_catch.groupby(by=['seasonal_year']).seasonal_day.min().reset_index()\
    .rename(columns={'seasonal_day': 'first_catch_day', 'seasonal_year': 'season'})

total_last_catch = total_daily_catch.groupby(by=['seasonal_year']).seasonal_day.max().reset_index()\
    .rename(columns={'seasonal_day': 'last_catch_day', 'seasonal_year': 'season'})

total_ice_season_length = total_last_catch.set_index('season').last_catch_day - total_first_catch.set_index('season').first_catch_day
total_ice_season_length = total_ice_season_length.reset_index(name='ice_season_length')

total_first_catch.to_csv('../data/total_first_catch.csv')
total_last_catch.to_csv('../data/total_last_catch.csv')
total_ice_season_length.to_csv('../data/total_ice_season_length.csv')

### First and Last Days, by Locality, 2012-2022

In [83]:
first_days = (
   local_daily_catch
   .groupby(by=['seasonal_year', 'sellers_locality'])
   .seasonal_day.min()
   .reset_index()
   .merge(
      local_daily_catch[['seasonal_year', 'sellers_locality', 'field_code', 'seasonal_day', 'landing_date', 'amount_in_kg']]
      )
   .rename(columns={'seasonal_day': 'catch_day', 'seasonal_year': 'season'})
) 

first_days['first_or_last'] = 'first'

last_days = (
   local_daily_catch
   .groupby(by=['seasonal_year', 'sellers_locality'])
   .seasonal_day.max()
   .reset_index()
   .merge(
      local_daily_catch[['seasonal_year', 'sellers_locality', 'field_code', 'seasonal_day', 'landing_date', 'amount_in_kg']]
      )
   .rename(columns={'seasonal_day': 'catch_day', 'seasonal_year': 'season'})
) 

last_days['first_or_last'] = 'last'

locality_seasons = (
    pd.concat([first_days, last_days])
    .sort_values(['season', 'sellers_locality'])
)

field_code_info = pd.read_csv("../data/fieldcodes.csv")

locality_seasons = (
    locality_seasons
    .merge(field_code_info, left_on = 'field_code', right_on = 'fieldcode')
    .drop(columns = 'fieldcode')
)

locality_seasons = locality_seasons[['season', 'sellers_locality',  'landing_date','first_or_last', 'catch_day', 'amount_in_kg', 'field_code', 'lat', 'lon']].sort_values(['season', 'sellers_locality'])

locality_seasons.to_csv("../data/locality_seasons.csv", index = None)


### Local First and Last Catch Days, 2012-2022

In [84]:
local_first_catch_dates = local_daily_catch.groupby(by=['seasonal_year', 'field_code', 'sellers_locality']).seasonal_day.min().reset_index()\
    .rename(columns={'seasonal_day': 'first_catch_day', 'seasonal_year': 'season'})

local_first_catch_dates.to_csv('../data/local_first_catch_dates.csv')

local_last_catch_dates = local_daily_catch.groupby(by=['seasonal_year', 'field_code', 'sellers_locality']).seasonal_day.max().reset_index()\
    .rename(columns={'seasonal_day': 'last_catch_day', 'seasonal_year': 'season'})

local_last_catch_dates.to_csv('../data/local_last_catch_dates.csv')

### Local Ice Season Length

In [89]:
local_ice_season_length = (local_last_catch.set_index(['season', 'field_code']).last_catch_day - local_first_catch.set_index(['season', 'field_code']).first_catch_day).reset_index(name='ice_season_length')

local_ice_season_length

,season,field_code,ice_season_length
0,2012,LN028,0
1,2012,LP028,63
2,2012,LP028,0
3,2012,LX025,0
4,2012,LX027,0
...,...,...,...
391,2021,MX009,9
392,2021,MZ008,18
393,2021,NA008,0
394,2021,NB008,6


In [ ]:
#todo
#derived
# Only consider fields that appear in every season
#fields_of_interest = set.intersection(*local_ice_season_length.groupby(by=['season']).field_code.unique().apply(lambda x: set(x)).values)


#local_ice_season_length[local_ice_season_length.field_code.isin(fields_of_interest)].groupby('field_code').corr(method='kendall').unstack()['season']['ice_season_length']

### Number of Fields Fished, 2012-2022

In [94]:
num_fields = local_first_catch.groupby('season').field_code.nunique().rename('n_fields').reset_index()

num_fields.to_csv('../data/num_fields.csv')

### Local Number of Fields Fished, 2012-2022

In [96]:
local_num_fields = ice_landings.groupby(by=['seasonal_year', 'sellers_locality']).field_code.nunique().reset_index().rename(columns={'field_code': 'n_fields'})
local_num_fields

local_num_fields.to_csv('../data/local_num_fields.csv')

### Fishing Grounds, by Locality, by Vessel Type, by Field Code, by Seasonal Year

In [101]:
fishing_grounds = landings.groupby(by=['seasonal_year', 'sellers_locality', 'vessel_type', 'field_code']).seller_id.nunique().reset_index().drop(columns='seller_id')

fishing_grounds['lon'] = fishing_grounds.field_code.map(lambda x: field_code_info[x]['lon'])
fishing_grounds['lat'] = fishing_grounds.field_code.map(lambda x: field_code_info[x]['lat'])

fishing_grounds.to_csv('../data/fishing_grounds.csv')

KeyError: 'LA019'